In [3]:
#rat
import numpy as np
#create a list of all names used
#names are saved in fale names, variable names
from events import RGB_TO_ACR, ACR_TO_PARENT
names = []
for v in RGB_TO_ACR:
    temp = [v, RGB_TO_ACR[v]]
    if RGB_TO_ACR[v] in ACR_TO_PARENT:
        temp.append(ACR_TO_PARENT[RGB_TO_ACR[v]]) 
    names.append(temp)

#create a list of areas available for labels
parent_labels = []
for i in range(len(names)):
    k = len(names[i])
    parent_labels.append(names[i][k-1]) 
parent_labels1 = np.array(parent_labels) # parent_labels1 ia a list of all available parent labels in this atlas

print (np.unique(parent_labels1))
    



['HBr' 'MBr' '[-]' '[Forebrain]' '[brainstem]' '[div. dev.]' '[pallium]'
 '[subpallium]' '[ventr]' '[white mat.]']


# Create file variables.py to collect all paths for each slice separately

In [4]:
#rat
#create list of events on each slice
from template_container_rat import SVG_PATHS, BRAIN_REGIONS, BRAIN_SLICES
events = []
for key in BRAIN_REGIONS:
    val = BRAIN_REGIONS[key]
    slices = val.keys()
    for item in slices:
        events.append([key, item])
print (len(events))
print(events[:10])

754
[['FF007F', '0'], ['FF007F', '1'], ['FF007F', '2'], ['FF007F', '3'], ['FF007F', '4'], ['FF007F', '5'], ['FF007F', '6'], ['FF007F', '7'], ['FF007F', '8'], ['FF007F', '9']]


In [5]:
#rat
#add to file variables.py (slice#)_paths = events that can be present on each slice
ev_to_apply = []
for j in range(0,179):
    ev_to_apply.append([j])
    for i in range(len(events)):
        if str(j) == str(events[i][1]):
            ev_to_apply[j].append(events[i][0])
f0 = open("variables.py","a")
f0.write('events_by_slices=' +str(tuple(ev_to_apply))+' \n')
f0.close()
print((ev_to_apply[0]))
print((ev_to_apply[1]))
print((ev_to_apply[2]))
print((ev_to_apply[3]))
print((ev_to_apply[4]))
print((ev_to_apply[5]))

[0, 'FF007F']
[1, 'FF007F']
[2, 'FF007F']
[3, 'FF007F']
[4, 'FF007F']
[5, 'FF007F']


In [6]:
#rat
# create a list of paths for each slice

f1 = open('variables.py', 'a')
for i in range(len(ev_to_apply)):
    f1.write('slice_'+str(i)+'_paths={')
    for j in range(1, len(ev_to_apply[i])):
        ev = ev_to_apply[i][j]
        path = BRAIN_REGIONS[ev]
        if str(ev_to_apply[i][0]) in path:
            f1.write('"'+str(ev_to_apply[i][j])+'"'+':'+str(path[str(i)])+(', '))
    f1.write('} \n')
f1.close()


In [7]:

f0.close()
f1.close()

In [8]:
#rat
# add to file variables.py paths_by_slices = [[{event:paths}], ...]
# 0 item in the list is for 0 slice and so on.
paths_by_slices = []

for i in range(len(ev_to_apply)):
    paths_by_slices.append([])
    for j in range(1, len(ev_to_apply[i])):
        ev = ev_to_apply[i][j]
        path = BRAIN_REGIONS[ev]
        if str(ev_to_apply[i][0]) in path:
            paths_by_slices[i].append(tuple(path[str(i)]))
f2 = open('variables.py', 'a')
f2.write('paths_by_slices = '+str(tuple(paths_by_slices))+'\n')
f2.close()

In [9]:
f2.close()

# Transform .svg coordinates to pixel coordinates

In [2]:
#rat
#just for the test
#create proper some of the labels for the template #51 (52(914)) and save as .png in labels folder
from template_container_rat import SVG_PATHS, BRAIN_REGIONS, BRAIN_SLICES
templ_51 = [51, 'FF007F', 'AA0000', '00FF00', 'AA5500', 
            '7FFF00', 'AAAA00', '00AAAA', '00AA00', 'FF00FF', 'AA00AA', '552A00', 'AA0055',
            'FF7F00', '2A5500', '005500', '55AA00']
#create the list of paths for the template 51
list_of_paths = []
for i in range(1, len(templ_51)):
    x = BRAIN_REGIONS[templ_51[i]]
    y = x['51']
    list_of_paths.append(y)
#change the list_of_paths from list of lists to list of numbers
unlisted = []
for i in range(len(list_of_paths)):
    for j in range(len(list_of_paths[i])):
        unlisted.append(list_of_paths[i][j])
print(list_of_paths[:4])
print(unlisted[1])

  


[[436, 437], [431], [455, 456], [435]]
437


In [8]:
import os
from svglib.svglib import svg2rlg
from reportlab.graphics import renderPM
from PIL import Image

#transform SVG paths to .png and then to pixel coordinates

#indexes of pathes to be used for the template 51 are stored in variable unlisted
labels = []
for i in range(len(list_of_paths)):
    f = open("transform.svg","w+")
    f.write('<svg  width="13200" height="10800" xmlns="http://www.w3.org/2000/svg"> \n')
    for j in range(len(list_of_paths[i])):
        f.write('    <path d="'+SVG_PATHS[list_of_paths[i][j]]+'" stroke="red" fill="red"/> \n')
    f.write('</svg>') 
    f.close() 
    drawing = svg2rlg("transform.svg")
    renderPM.drawToFile(drawing, "labels/13"+str(i)+".png", fmt="PNG") 
    r = Image.open("labels/13"+str(i)+".png")
    r1=r.resize((400,330), Image.LANCZOS)
    r1.save("labels/r.png", 'PNG')
    r2 = Image.open('labels/r.png')
    r3=Image.new('RGBA', (340, 330), (0, 0, 0, 255))
    r3.paste(r2,(0, 0))
    r3.save("labels/"+str(i)+".png", 'PNG')
    r.close()
    r2.close()
    os.remove("labels/13"+str(i)+".png")
    os.remove("labels/r.png")
    coordinates = []
    width, height = r3.size
    for x in range(width):
        for y in range(height):
            r,g,b,a = r3.getpixel((x,y))
            if r==255 and g==b==0:
                coordinates.append((x,y))
    labels.append(tuple(coordinates))
    r3.close
    os.remove("labels/"+str(i)+".png")
print(len(labels))
print(len(labels[1]))
    

16
17823


In [6]:
r.close()

AttributeError: 'int' object has no attribute 'close'

In [9]:
#rat
#test if converted pixel coordinates correspond to the real label (white matter, slice(176))
from PIL import ImageFilter
im = Image.new('RGBA', (340, 330), (0, 0, 0, 0))
for j in range (len(labels)):
    for i in range(len(labels[j])):
        pixel=labels[j][i]
        pixel_w=int(pixel[0])
        pixel_h=int(pixel[1])
        im.putpixel(pixel, (255,0,0,255))
    im = im.filter(ImageFilter.GaussianBlur(radius=1))
    im.save('labels/im'+str(j)+'.png', 'PNG')



# create .py file: labels.py in a container folder

In [10]:
#rat
import os
from template_container_rat import SVG_PATHS
import time
from variables import events_by_slices, paths_by_slices
from svglib.svglib import svg2rlg
from reportlab.graphics import renderPM
from PIL import Image
start_time = time.time()

for i in range(len(paths_by_slices)):
    labels_coordinates = open('labels/slice_'+str(events_by_slices[i][0])+'.py', 'a')
    for j in range(len(paths_by_slices[i])):
        paths_pos = paths_by_slices[i][j]
        labels_coordinates.write('coordinates_'+str(events_by_slices[i][j+1])+' = (')
        f = open("transform.svg","w+")
        f.write('<svg  width="13200" height="10800" xmlns="http://www.w3.org/2000/svg"> \n')
        for k in range(len(paths_pos)):
            f.write('    <path d="'+SVG_PATHS[paths_pos[k]]+'" stroke="red" fill="red"/> \n')
        f.write('</svg>') 
        f.close() 
        drawing = svg2rlg("transform.svg")
        renderPM.drawToFile(drawing, "labels/temp"+str(i)+".png", fmt="PNG") 
        r = Image.open("labels/temp"+str(i)+".png")
        r1=r.resize((400,330), Image.LANCZOS)
        r1.save("labels/r.png", 'PNG')
        r2 = Image.open('labels/r.png')
        r3=Image.new('RGBA', (340, 330), (0, 0, 0, 255))
        r3.paste(r2,(0, 0))
        r3.save("labels/"+str(i)+".png", 'PNG')
        r.close()
        r2.close()
        os.remove("labels/temp"+str(i)+".png")
        os.remove("labels/r.png")
        coordinates = []
        width, height = r3.size
        m=0
        for x in range(width):
            for y in range(height):
                r,g,b,a = r3.getpixel((x,y))
                if r==255 and g==b==0:
                    m = m+1
                    coordinates.append((x,y))
        t = tuple(coordinates)
        for a in range(len(t)):
            labels_coordinates.write(str(t[a])+', ')
            if a%72 == 0:
                labels_coordinates.write(" \n")
        labels_coordinates.write(") \n")
        r3.close
        os.remove("labels/"+str(i)+".png")
    labels_coordinates.close()
print("--- %s seconds ---" % (time.time() - start_time))





--- 5672.528589248657 seconds ---


In [11]:
labels_coordinates.close()
r.close()

AttributeError: 'int' object has no attribute 'close'

In [2]:
#dictionalry to create corresponding slices' names
#rat
list_1 = []
slices_numbers = {}
i = 1424
for k in range(0, 135):
    slices_numbers[i] = k
    i = i-10
print(slices_numbers)

{1424: 0, 1414: 1, 1404: 2, 1394: 3, 1384: 4, 1374: 5, 1364: 6, 1354: 7, 1344: 8, 1334: 9, 1324: 10, 1314: 11, 1304: 12, 1294: 13, 1284: 14, 1274: 15, 1264: 16, 1254: 17, 1244: 18, 1234: 19, 1224: 20, 1214: 21, 1204: 22, 1194: 23, 1184: 24, 1174: 25, 1164: 26, 1154: 27, 1144: 28, 1134: 29, 1124: 30, 1114: 31, 1104: 32, 1094: 33, 1084: 34, 1074: 35, 1064: 36, 1054: 37, 1044: 38, 1034: 39, 1024: 40, 1014: 41, 1004: 42, 994: 43, 984: 44, 974: 45, 964: 46, 954: 47, 944: 48, 934: 49, 924: 50, 914: 51, 904: 52, 894: 53, 884: 54, 874: 55, 864: 56, 854: 57, 844: 58, 834: 59, 824: 60, 814: 61, 804: 62, 794: 63, 784: 64, 774: 65, 764: 66, 754: 67, 744: 68, 734: 69, 724: 70, 714: 71, 704: 72, 694: 73, 684: 74, 674: 75, 664: 76, 654: 77, 644: 78, 634: 79, 624: 80, 614: 81, 604: 82, 594: 83, 584: 84, 574: 85, 564: 86, 554: 87, 544: 88, 534: 89, 524: 90, 514: 91, 504: 92, 494: 93, 484: 94, 474: 95, 464: 96, 454: 97, 444: 98, 434: 99, 424: 100, 414: 101, 404: 102, 394: 103, 384: 104, 374: 105, 364: 1

In [10]:
#f1.close()
f0.close()
f2.close()

# Test created labels

In [4]:
#slice 57 (854 in the folder)
import labels.slice_57 as slice_57
from PIL import Image, ImageFilter
labels = [slice_57.coordinates_FF007F, slice_57.coordinates_AA0000, slice_57.coordinates_AA5500, slice_57.coordinates_7FFF00,
         slice_57.coordinates_AAAA00, slice_57.coordinates_00AAAA, slice_57.coordinates_FF00FF, slice_57.coordinates_AA00AA,
         slice_57.coordinates_552A00, slice_57.coordinates_FF7F00, slice_57.coordinates_2A5500, slice_57.coordinates_005500,
         slice_57.coordinates_55AA00, slice_57.coordinates_FFFF00]

temp = Image.open('slice_numbers/854.png')
templ = temp.convert('RGBA')
templ1 = Image.new('RGBA', (340,330), (0, 0, 0, 0))
for i in range(len(labels)):
    event = labels[i]
    for j in range(len(event)):
        pixel = event[j]
        pixel_w = int(pixel[0])
        pixel_h = int(pixel[1])
        templ1.putpixel((pixel_w, pixel_h), (255,0,0,255))    
templ1 = templ1.filter(ImageFilter.GaussianBlur(radius=2))

templ.paste(templ1,(0, 0), mask = templ1)
templ1.save('00slice_57.png', 'PNG')
templ.save('slice_57.png', 'PNG')

In [3]:
img = Image.open('slice_numbers/854.png')
print(img.format)

JPEG
